<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Análise-da-distribuição-dos-locais-de-tratamento" data-toc-modified-id="Análise-da-distribuição-dos-locais-de-tratamento-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Análise da distribuição dos locais de tratamento</a></span><ul class="toc-item"><li><span><a href="#Propondo-novos-centros-de-tratamento" data-toc-modified-id="Propondo-novos-centros-de-tratamento-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Propondo novos centros de tratamento</a></span></li></ul></li></ul></div>

# Análise da distribuição dos locais de tratamento

Nesse notebook iremos analisar a distribuição dos locais de tratamento dentro de um estado específico e vamos analisar como um novo centro de tratamento diminuiría a distância que as pessoas devem percorrer até os locais de tratamento.

In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

from geopy.distance import geodesic

import mplleaflet
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

import os
import warnings

from k_medoids import KMedoidsWeighted
import helper as hp

warnings.simplefilter("ignore")
tqdm.pandas()

np.random.seed(3)

In [2]:
data_dir = '../data/' #diretorio com os dados
quimio_name = 'Pulmão Quimioterapia SIA-SUS.csv' #nome do arquivo de quimioterapia
radio_name = 'Pulmão Radioterapia SIA-SUS.csv' #nome do arquivo de radioterapia
mortalidade_name = 'Mortalidade Pulmão SIM-SUS.csv' #nome do arquivo de mortalidade
municipios_name = 'dados_municipios.csv' #nome do arquivo com os dados dos municipios

usecols = ['AP_UFMUN', 'AP_MUNPCN']

dtype_cols = {'AP_MUNPCN': str, 'AP_UFMUN': str}

codigo_uf = '25' #estado a ser analisado

In [3]:
# Lendo os dataframes:

quimio = pd.read_csv(os.path.join(data_dir, quimio_name), encoding='latin', 
                     error_bad_lines=False, low_memory=False, usecols=usecols, dtype=dtype_cols)

radio = pd.read_csv(os.path.join(data_dir, radio_name), encoding='latin',
                    error_bad_lines=False, low_memory=False, usecols=usecols, dtype=dtype_cols)

municipios = pd.read_csv(os.path.join(data_dir, municipios_name), index_col=0, dtype={'COD_MUN': str})

Os códigos IBGE dos municípios nas APACs têm 6 dígitos enquanto que os códigos nos dados sobre os municípios têm 7 dígitos, iremos remover o últimio dígito no dataset sobre os municípios

In [4]:
municipios.index = municipios['COD_MUN'].apply(lambda x: x[:-1])

In [5]:
#vamos juntar os dataframes de quimioterapia e radioterapia criando uma coluna de tratamento
quimio['TRAT'] = 'quimio'
radio['TRAT'] = 'radio'
cancer = pd.concat([quimio, radio])

In [6]:
#iremos agora calcular a distância da cidade do paciente a cidade de tratamento
cancer['DIST'] = cancer.progress_apply(lambda row: hp.calculte_distance(row, municipios), axis=1)

100%|██████████| 330771/330771 [01:10<00:00, 4673.23it/s]


Nossa análise será baseada apenas em um estado específico

In [7]:
cancer_estado = hp.generate_df_estado(cancer, municipios, codigo_uf)

In [8]:
fig, ax = plt.subplots(figsize=[10,10])
ax = sns.scatterplot(data=cancer_estado.groupby(['LON_PCN', 'LAT_PCN']).size().reset_index(), x='LON_PCN', y='LAT_PCN', color='b', ax=ax)
ax = sns.scatterplot(data=cancer_estado.groupby(['LON_MUN', 'LAT_MUN']).size().reset_index(), x='LON_MUN', y='LAT_MUN', color='r', ax=ax, s=200)
mplleaflet.display(fig=fig)

O gráfico acima mostra todas as cidades do estado como pontos azuis e as cidades em que há tratamento em vermelho.

In [23]:
temp_df.columns

Index(['250400', '250750'], dtype='object', name='AP_UFMUN')

In [24]:
medoids_idxs

[0, 1, 2, 3]

In [26]:
cancer_estado['AP_MUNPCN'].value_counts().head(4)

250750    1477
250400     684
251080     173
251620     143
Name: AP_MUNPCN, dtype: int64

In [30]:
medoids_codigos = [cancer_estado['AP_MUNPCN'].value_counts().index[i] for i in medoids_idxs]

In [31]:
medoids_codigos

['250750', '250400', '251080', '251620']

In [32]:
norm = mpl.colors.Normalize(vmin=0, vmax=len(medoids_codigos))
cmap = cm.Dark2
m = cm.ScalarMappable(norm=norm, cmap=cmap)

In [40]:
fig, ax = plt.subplots(figsize=[15,15])

#Onde as pessoas daquela cidade foram se tratar
temp_df = pd.crosstab(index=cancer_estado['AP_MUNPCN'], columns=cancer_estado['AP_UFMUN'])
#Quantidade de ocorrencias na cidade
tamanho_municipio = cancer_estado['AP_MUNPCN'].value_counts()


colors = {col: m.to_rgba(i) for i, col in enumerate(medoids_codigos)}

plt.figure(figsize=[12,12])
for mun in tqdm(temp_df.index):
    lat, long = hp.coord_from_codigo(mun, municipios)
    if lat==0:
        continue
    if mun in temp_df.columns:   
        ax.scatter(long, lat, s=400, color=colors[temp_df.loc[mun].argmax()], marker='^')
    else:
        ax.scatter(long, lat, color=colors[temp_df.loc[mun].argmax()], s=100)
        


mplleaflet.display(fig=fig)

100%|██████████| 160/160 [00:00<00:00, 455.76it/s]


<Figure size 864x864 with 0 Axes>

No mapa podemos ver onde as pessoas deuma cidade específica mais vão fazer o tratamento do câncer.

Vamos calcular a distância média que uma pessoa de outra cidade tem que percorre para se tratar na condição atual e quantas pessoas são atendidas em cada cidade em que há tratamento.

In [10]:
print("Distância média percorrida por pessoas que não estão na cidade de tratamento: {:.2f}km".format(cancer_estado[cancer_estado['DIST']>0]['DIST'].mean()))

Distância média percorrida por pessoas que não estão na cidade de tratamento: 197.40km


In [11]:
for nome in temp_df.columns:
    print("{} --> {} pacientes".format(municipios.loc[nome]['NOME'], temp_df.sum().loc[nome]))

Campina Grande --> 1524 pacientes
João Pessoa --> 3849 pacientes


## Propondo novos centros de tratamento

Como podemos notar pelos mapas, há, na maioria dos estados brasileiros, uma má distribuição dos locais de tratamento em relação a população. Dessa forma, criamos uma nova implementação do algoritimo K medoids para lidar com pesos nos pontos de entrada, a classe está implementada no arquivo k_medoids.py

In [12]:
X = [list(hp.coord_from_codigo(codigo, municipios)) for codigo in cancer_estado['AP_MUNPCN'].value_counts().index]
weights = cancer_estado['AP_MUNPCN'].value_counts()

In [13]:
X_weighted = [x+[w] for x,w in zip(X, weights)]

In [14]:
#quantidade inicial de centros de tratamento
n_tratamento = len(temp_df.columns)
#quantidacde de novos centros
n_novos = 2
clustering = KMedoidsWeighted(n_clusters=n_tratamento+n_novos)

In [15]:
medoids_idxs, clusters = clustering.fit(X_weighted, max_itr=10, verbose=1)

Iteration: 0 ... loss: 2569.6546557831302
Iteration: 1 ... loss: 1337.95161776012
Iteration: 2 ... loss: 1273.6068157526922


In [41]:
fig, ax = plt.subplots(figsize=[15,15])

for cod, cluster in zip(medoids_codigos,clusters):
    for p in cluster:
        if p in medoids_idxs:
            plt.scatter(X[p][1], X[p][0], color=colors[cod], s=300, marker='^')
        else:
            plt.scatter(X[p][1], X[p][0], color=colors[cod])
            
 
fig.set_dpi(100)
mplleaflet.display(fig=fig)

In [19]:
tot_dist = 0
tot_pop = 0
for medoid, cluster in zip(medoids_idxs, clusters):
    for p in cluster:
        if p!=medoid:
            pop = X_weighted[p][2]
            lat1, long1 = X_weighted[p][0], X_weighted[p][1]
            lat2, long2 = X_weighted[medoid][0], X_weighted[medoid][1]
            tot_dist += geodesic((lat1, long1),(lat2, long2)).km*pop
            tot_pop += pop
        
mean_new_dist = tot_dist/tot_pop

In [20]:
print("Distância média percorrida por pessoas que não estão na cidade de tratamento com a nova configuração: {:.2f}km".format(mean_new_dist))

Distância média percorrida por pessoas que não estão na cidade de tratamento com a nova configuração: 55.13km


Como podemos notar houve uma diminuição significativa da distância média percorrida